In [47]:
import numpy as np
import meshplot as mp
import results.runs_sdf as runs
import os
import matplotlib.pyplot as plt
import torch
import model.model_sdf as sdf_model
import trimesh
import results
from examples import reconstruct_all_from_latent
from utils import utils_deepsdf
import results

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Visualize mesh
First optimised latent code during training:

In [58]:
latent_code_idx = 230
folder = '10_03_130134'
resolution = 50
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

run_dir = os.path.join(os.path.dirname(runs.__file__), run_folder)
weights_path = os.path.join(run_dir, 'weights.pt')
results_path = os.path.join(run_dir, 'results.npy')
results_dict = np.load(results_path, allow_pickle=True).item()
latent = torch.tensor(results_dict['train']['best_latent_codes'][latent_code_idx])

model = sdf_model.SDFModelMulti(num_layers=8, no_skip_connections=False).to(device)

# Load weights
#weights_path = os.path.join(os.path.dirname(results.__file__), 'runs_sdf', folder, 'weights.pt')
model.load_state_dict(torch.load(weights_path, map_location=device))

# Load results dictionary
#results_dict_path = os.path.join(os.path.dirname(results.__file__), 'runs_sdf', folder, 'results.npy')
#results_dict = np.load(results_dict_path, allow_pickle=True).item()

latent_codes_list = reconstruct_all_from_latent.get_latent_codes_training(results_dict)[latent_code_idx]

coords, grad_size_axis = utils_deepsdf.get_volume_coords(resolution)
coords = coords.clone().to(device)
coords_batches = torch.split(coords, 100000)

sdf = utils_deepsdf.predict_sdf(latent, coords_batches, model)
try:
    vertices, faces = utils_deepsdf.extract_mesh(grad_size_axis, sdf)
except:
    print('Exception triggered')
    exit()
    
# save mesh using meshplot
mesh_dir = os.path.join(run_dir, 'meshes_training')
if not os.path.exists(mesh_dir):
    os.mkdir(mesh_dir)
mesh_path = os.path.join(mesh_dir, f'latent_{latent_code_idx}.html')
utils_deepsdf.save_meshplot(vertices, faces, mesh_path)

# save mesh as obj
obj_path = os.path.join(mesh_dir, f"mesh_{latent_code_idx}.obj")
trimesh.exchange.export.export_mesh(trimesh.Trimesh(vertices, faces), obj_path, file_type='obj')

Plot saved to file /Users/ri21540/Documents/PhD/Code/DeepSDF/results/runs_sdf/10_03_130134/meshes_training/latent_230.html.


'# https://github.com/mikedh/trimesh\nv 29.99788857 38.00000000 37.00000000\nv 30.00000000 37.97860718 37.00000000\nv 30.00000000 38.00000000 36.99205780\nv 30.00000000 38.00000000 37.10487366\nv 29.99504280 39.00000000 37.00000000\nv 30.00000000 39.00000000 36.97505188\nv 30.00000000 38.79317474 38.00000000\nv 29.99520111 39.00000000 38.00000000\nv 30.00000000 39.00000000 38.02776718\nv 29.99475479 40.00000000 37.00000000\nv 30.00000000 40.00000000 36.98059082\nv 29.99525070 40.00000000 38.00000000\nv 30.00000000 40.00000000 38.03466415\nv 29.99457932 41.00000000 37.00000000\nv 30.00000000 41.00000000 36.98058319\nv 29.99490929 41.00000000 38.00000000\nv 30.00000000 41.00000000 38.03588486\nv 29.99480629 42.00000000 37.00000000\nv 30.00000000 42.00000000 36.98088837\nv 29.99505424 42.00000000 38.00000000\nv 30.00000000 42.00000000 38.03430939\nv 29.99472618 43.00000000 37.00000000\nv 30.00000000 43.00000000 36.97558212\nv 29.99628258 43.00000000 38.00000000\nv 30.00000000 43.00000000 